In [ ]:
import pandas as pd
from google.colab import drive

# =========================================================
# 1. Montar Google Drive
# =========================================================
# Esto te pedirá que autorices el acceso a tu Google Drive
# y que copies y pegues un código de verificación.
try:
    drive.mount('/content/drive')
    print("✅ Google Drive montado correctamente.")
except Exception as e:
    print(f"❌ Error al montar Google Drive: {e}")

# =========================================================
# 2. Definir la Ruta y Cargar el DataFrame
# =========================================================

# ¡IMPORTANTE! DEBES CAMBIAR ESTA RUTA a la ubicación real
# de tu archivo 'Proyecto_final.csv' dentro de tu Drive.
# Ejemplo: si está en una carpeta llamada 'Datos', la ruta sería:
ruta_archivo = '/content/drive/MyDrive/Proyecto_final.csv'

try:
    # Cargar la base de datos: 'sep=;' para columnas y 'decimal=,' para números
    df = pd.read_csv(ruta_archivo, sep=';', decimal=',')

    print(f"\n✅ Archivo cargado correctamente desde: {ruta_archivo}")
    print(f"Dimensiones del DataFrame (filas, columnas): {df.shape}")
    print("\nPrimeras 5 filas del DataFrame:")
    display(df.head())

except FileNotFoundError:
    print(f"\n❌ ERROR: Archivo no encontrado. Por favor, verifica la ruta: '{ruta_archivo}'")
except Exception as e:
    print(f"\n❌ ERROR al cargar el archivo. Verifica los separadores: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado correctamente.

✅ Archivo cargado correctamente desde: /content/drive/MyDrive/Proyecto_final.csv
Dimensiones del DataFrame (filas, columnas): (103, 145)

Primeras 5 filas del DataFrame:


,Id,Incorporacion,Formulario,Horadeinicio,Horadefinalización,HoraRealizacion,TiempoRequerido,CI,Sexo,Edad,...,GBC_21,GBC_22,GBC_23,GBC_24,GBC_25,GBC_26,GBC_27,GBC_28,GBC_29,GBC_Puntaje
0,1,0,1,5/1/2025 22:00,5/1/2025 22:13,22.0,13.32,1,1,999,...,1,1,1,2,2,1,1,1,0,NaN
1,2,1,1,9/1/2025 17:10,9/1/2025 17:34,17.0,23.80,1,0,56,...,2,1,1,2,1,1,0,1,0,NaN
2,3,1,1,9/1/2025 18:05,9/1/2025 18:34,18.0,29.57,1,0,51,...,2,2,2,1,2,2,1,0,1,NaN
3,4,1,1,9/1/2025 18:03,9/1/2025 18:45,18.0,41.90,1,0,40,...,2,1,0,0,0,2,1,0,2,NaN
4,5,1,1,9/1/2025 18:35,9/1/2025 18:58,18.0,22.20,1,0,42,...,2,1,2,0,1,2,2,1,1,NaN


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

# =========================================================
# 1. Preparación y Limpieza de Datos
# =========================================================

# NOTA: ASUMO que el DataFrame 'df' (cargado de Proyecto_final.csv) está disponible.

# A. Limpieza de separador decimal y '999'
# Se utiliza una copia para no alterar el original
df_model = df.copy()

# 1. Corrección del separador decimal (si no se hizo en la carga inicial)
# Dado que confirmaste que la carga ya se hizo, este paso asegura que los floats estén bien
# Si la carga inicial no tuvo decimal=',', estas columnas serían tipo object.
# Si ya son float, esta conversión no hace daño.
float_cols = ['TiempoRequerido', 'HoraRealizacion']
for col in float_cols:
    if df_model[col].dtype == 'object':
        df_model[col] = df_model[col].str.replace(',', '.', regex=False).astype(float)


# 2. Reemplazar '999' por NaN en variables clave continuas
cols_to_clean_999 = ['Edad', 'AñosPractica', 'HorasTrabajoSemanal', 'TiempoEnViaje']
df_model[cols_to_clean_999] = df_model[cols_to_clean_999].replace(999, np.nan)


# B. Selección de Variables
y_variable = 'Puntaje_Habitos'

# Variables Independientes (X)
# Cruzo las variables más relevantes: datos demográficos, carga laboral y una variable de escala (MBI_Burnout)
X_cols = ['Edad', 'AñosPractica', 'HorasTrabajoSemanal', 'TiempoRequerido',
          'Sexo', 'EstadoCivil', 'Guardia', 'MBI_Burnout']

df_model = df_model[X_cols + [y_variable]].copy()

# C. Manejo de Variables Categóricas y NaN
# 1. Codificación One-Hot para variables nominales
# 'drop_first=True' evita la multicolinealidad perfecta (dummy variable trap)
df_model = pd.get_dummies(df_model, columns=['Sexo', 'EstadoCivil', 'Guardia'], drop_first=True)

# 2. Eliminación de filas con cualquier NaN para asegurar un conjunto de datos limpio para el modelo
df_model.dropna(subset=df_model.columns, inplace=True)

# Convert boolean columns to integer (0 or 1) for statsmodels
for col in df_model.columns:
    if df_model[col].dtype == bool:
        df_model[col] = df_model[col].astype(int)


print(f"✅ Dataframe listo. Dimensiones finales: {df_model.shape}")

# =========================================================
# 2. Regresión Lineal Múltiple y Evaluación
# =========================================================

# Definición final de X e y
y = df_model[y_variable]

# Excluir la variable dependiente para obtener las predictoras
X_cols_final = [col for col in df_model.columns if col != y_variable]

# Agregar la constante (Intercepto) para el modelo OLS
X = sm.add_constant(df_model[X_cols_final])

# Separación de datos en conjuntos de entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo OLS (Mínimos Cuadrados Ordinarios)
model = sm.OLS(y_train, X_train).fit()

# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# =========================================================
# 3. Eficiencia del Modelo (Métricas Clave)
# =========================================================

# 1. Resumen Detallado de Statsmodels (contiene R2, F-Stat, y P-valores)
print("\n=========================================================")
print("  RESUMEN OLS DEL MODELO (Análisis Estadístico) ")
print("=========================================================")
print(model.summary())

# 2. Métricas de Error en el conjunto de prueba (Test)
print("\n=========================================================")
print("  MÉTRICAS DE EFICIENCIA EN DATOS DE PRUEBA ")
print("=========================================================")
# R² (Coeficiente de Determinación)
r2 = r2_score(y_test, y_pred)
# RMSE (Raíz del Error Cuadrático Medio)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Variable Dependiente (y): {y_variable}")
print(f"R² (Coeficiente de Determinación) en Test: {r2:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio) en Test: {rmse:.4f}")

✅ Dataframe listo. Dimensiones finales: (101, 14)

  RESUMEN OLS DEL MODELO (Análisis Estadístico) 
                            OLS Regression Results                            
Dep. Variable:        Puntaje_Habitos   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     1.616
Date:                Sat, 04 Oct 2025   Prob (F-statistic):              0.120
Time:                        21:53:24   Log-Likelihood:                -224.39
No. Observations:                  80   AIC:                             470.8
Df Residuals:                      69   BIC:                             497.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------

/usr/local/lib/python3.12/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import statsmodels.api as sm

# =========================================================
# 1. Preparación y Limpieza de Datos (para Regresión Logística)
# =========================================================

# ASUMO que el DataFrame 'df' (cargado de Proyecto_final.csv con decimal=,) está disponible.

df_model = df.copy()

# A. Limpieza de '999' (reemplazar por NaN) en variables continuas y categóricas
cols_to_clean_999_continuous = ['Edad', 'AñosPractica', 'HorasTrabajoSemanal', 'TiempoEnViaje']
cols_to_clean_999_categorical = ['Sexo', 'EstadoCivil', 'Guardia'] # Also treat 999 as NaN in categorical

df_model[cols_to_clean_999_continuous + cols_to_clean_999_categorical] = df_model[cols_to_clean_999_continuous + cols_to_clean_999_categorical].replace(999, np.nan)


# B. Definición de X e y
y_variable = 'MBI_Burnout' # Variable Categórica a predecir (0 o 1)

# Variables Predictoras (X): Selecciono un subconjunto para reducir multicolinealidad
# Removed categorical variables and MBI_Burnout for simplicity
X_cols = ['Edad', 'AñosPractica', 'HorasTrabajoSemanal', 'TiempoRequerido', 'Puntaje_Habitos']


df_model = df_model[X_cols + [y_variable]].copy()

# E. Eliminación de filas con cualquier NaN para asegurar un conjunto de datos limpio para el modelo
# This step is moved up to remove NaNs (including those from '999') before one-hot encoding
df_model.dropna(subset=df_model.columns, inplace=True)


# C. Conversión a Binario: Asegurarse de que y sea binaria (0 o 1)
# Si MBI_Burnout tiene más de 2 niveles, se debe elegir uno. Asumiremos que 1=Sí/Alto Burnout y 0=No/Bajo.
# En algunos datasets, se puede remapear (ej: 'Alto' -> 1, 'Bajo'/'Medio' -> 0).
# Usamos .astype(int) si ya tiene valores numéricos que representan las clases.


# D. Codificación One-Hot para X categórica
# Now that NaNs are handled, one-hot encoding will not create '999' dummy variables
# Removed categorical columns from X_cols, so no one-hot encoding needed here
# df_model = pd.get_dummies(df_model, columns=['Sexo', 'EstadoCivil', 'Guardia'], drop_first=True)

# Convert boolean columns to integer (0 or 1) for statsmodels (this might be redundant now but is safe)
for col in df_model.columns:
    if df_model[col].dtype == bool:
        df_model[col] = df_model[col].astype(int)

# Drop columns with zero variance to avoid multicollinearity issues (should not be needed now but is safe)
zero_variance_cols = df_model.columns[df_model.var() == 0]
df_model.drop(columns=zero_variance_cols, inplace=True)


print(f"✅ Dataframe listo para Regresión Logística. Dimensiones: {df_model.shape}")

# =========================================================
# 2. Regresión Logística y Evaluación
# =========================================================

# Definición final de X e y
y = df_model[y_variable].astype(int) # Aseguramos que 'y' sea binaria (int)
X_cols_final = [col for col in df_model.columns if col != y_variable]
X = sm.add_constant(df_model[X_cols_final]) # Añadir constante/intercepto

# Separación de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo Logit (Regresión Logística)
# Increased maxiter as suggested by the warning
logit_model = sm.Logit(y_train, X_train).fit(maxiter=100)

# Predicciones en el conjunto de prueba
# Se usa predict(..., cutoff=0.5) para obtener la clase (0 o 1)
y_pred_proba = logit_model.predict(X_test)
y_pred_class = (y_pred_proba >= 0.5).astype(int)

# =========================================================
# 3. Eficiencia del Modelo (Métricas Clave)
# =========================================================

print("\n=========================================================")
print("     RESUMEN DEL MODELO LOGIT (Regresión Logística) ")
print("=========================================================")
print(logit_model.summary())

print("\n=========================================================")
print("      MÉTRICAS DE EFICIENCIA EN DATOS DE PRUEBA ")
print("=========================================================")
# Exactitud (Accuracy)
accuracy = accuracy_score(y_test, y_pred_class)
# Reporte de Clasificación (Precisión, Recall, F1-Score)
report = classification_report(y_test, y_pred_class, zero_division=0)
# Matriz de Confusión
conf_mat = confusion_matrix(y_test, y_pred_class)

print(f"Variable Dependiente (y): {y_variable}")
print(f"Exactitud (Accuracy) en Test: {accuracy:.4f}\n")
print(report)
print("\nMatriz de Confusión:\n", conf_mat)

✅ Dataframe listo para Regresión Logística. Dimensiones: (101, 6)
Optimization terminated successfully.
         Current function value: 0.415495
         Iterations 9

     RESUMEN DEL MODELO LOGIT (Regresión Logística) 
                           Logit Regression Results                           
Dep. Variable:            MBI_Burnout   No. Observations:                   80
Model:                          Logit   Df Residuals:                       74
Method:                           MLE   Df Model:                            5
Date:                Sat, 04 Oct 2025   Pseudo R-squ.:                 0.06376
Time:                        21:58:06   Log-Likelihood:                -33.240
converged:                       True   LL-Null:                       -35.503
Covariance Type:            nonrobust   LLR p-value:                    0.4762
                          coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

In [ ]:
# Conclusiones sobre la Eficiencia y Variables
# A. Mejor Variable Dependiente: MBI_Burnout
# Razón: MBI_Burnout (o el nivel de Burnout) es la variable de resultado clave en el ámbito de la salud laboral. Predecir esta variable permite identificar qué factores (edad, carga laboral, hábitos, etc.) aumentan o disminuyen la probabilidad de que un individuo sufra de Burnout.

# B. Valores de Eficiencia del Modelo Logístico
# Dado que es un modelo de clasificación, las métricas clave son:

# Exactitud (Accuracy):

# Indica el porcentaje total de predicciones correctas (casos de Burnout y casos sin Burnout). Es la métrica más simple.

# Pseudo−R2
#  (del resumen Logit):

#Es el análogo del R2
#  en la Regresión Logística. El valor de Pseudo R-sq. (ej. McFadden R-squared) indica qué tan bien el modelo explica la variabilidad en MBI_Burnout. Un valor entre 0.2 y 0.4 es generalmente considerado muy bueno para datos de ciencias sociales.

# Precisión (Precision) y Sensibilidad (Recall):

# Precisión (Clase 1, Burnout): De todas las veces que el modelo predijo "Burnout", ¿cuántas veces acertó? (Evita falsos positivos).

# Sensibilidad (Clase 1, Burnout): De todos los casos reales de "Burnout" que existían, ¿cuántos pudo identificar correctamente el modelo? (Evita falsos negativos).

# C. Mejores Variables Predictoras (P-valores)
# Para saber qué variables son las más importantes en la predicción de Burnout:

# Observa la columna P>|z| (P-valor) en el resumen del modelo Logit.

# Cualquier variable (ej. HorasTrabajoSemanal, Puntaje_Habitos) con un P-valor menor a 0.05 es un predictor estadísticamente significativo de la probabilidad de que ocurra el Burnout.

# El coeficiente (coef) asociado te dirá si la variable tiene una relación positiva (aumenta la probabilidad de Burnout) o negativa (disminuye la probabilidad).